## 통계청 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import re 
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
df = pd.read_csv('./용도별_전력사용량.csv', encoding='CP949')

In [3]:
df

,월별(1),2013,2013.1,2013.2,2013.3,2013.4,2013.5,2013.6,2013.7,2013.8,...,2020.6,2020.7,2020.8,2020.9,2020.10,2020.11,2020.12,2020.13,2020.14,2020.15
0,월별(1),합계,합계,가정용,가정용,공공용,공공용,서비스업,서비스업,산업용,...,서비스업,서비스업,산업용,산업용,산업용,산업용,산업용,산업용,산업용,산업용
1,월별(1),소계,소계,소계,소계,소계,소계,소계,소계,소계,...,소계,소계,소계,소계,농림수산업,농림수산업,광업,광업,제조업,제조업
2,월별(1),전력사용량 (MWh),점유율 (%),전력사용량 (MWh),점유율 (%),전력사용량 (MWh),점유율 (%),전력사용량 (MWh),점유율 (%),전력사용량 (MWh),...,전력사용량 (MWh),점유율 (%),전력사용량 (MWh),점유율 (%),전력사용량 (MWh),점유율 (%),전력사용량 (MWh),점유율 (%),전력사용량 (MWh),점유율 (%)
3,합계,45466410,100.00,2448504,5.39,1026522,2.26,6399018,14.07,35592367,...,7423314,14.72,38899147,77.15,2043578,4.05,198914,0.39,36656655,72.70
4,1월,4111542,100.00,225206,5.48,117921,2.87,769232,18.71,2999184,...,780585,17.07,3416415,74.73,180379,3.95,13989,0.31,3222047,70.48
5,2월,3622215,100.00,222551,6.14,104851,2.89,730022,20.15,2564791,...,758705,17.33,3255615,74.38,180763,4.13,14462,0.33,3060390,69.92
6,3월,3808272,100.00,192539,5.06,94908,2.49,603436,15.85,2917389,...,680584,15.41,3403793,77.09,166033,3.76,15856,0.36,3221904,72.97
7,4월,3708180,100.00,202949,5.47,90246,2.43,577898,15.58,2837087,...,605702,14.87,3145326,77.23,145620,3.58,15303,0.38,2984403,73.28
8,5월,3659870,100.00,190417,5.20,74324,2.03,483109,13.20,2912019,...,554291,14.14,3066778,78.23,146745,3.74,13862,0.35,2906171,74.14
9,6월,3581181,100.00,189332,5.29,70570,1.97,406880,11.36,2914399,...,540258,13.34,3197748,78.96,160843,3.97,14541,0.36,3022364,74.63


In [4]:
def term_useage(df, start_year_month, end_year_month):
    month = df['월별(1)']
    
    df.columns = [row.split('.')[0] for row in df.columns.astype(str)]
    df = df.drop(['월별(1)'], axis=1)

    # total dataframe preprocessing
    power_use = pd.DataFrame()

    for year in tqdm(df.columns.unique()):
        df_year = df[str(year)]
        df_year['month'] = month
        
        df_year = df_year.set_index('month').iloc[:,0:16].drop(['합계'], axis=0)
        df_year['year'] = year
        df_year = df_year.reset_index()
    
        # drop '점유율 (%)'
        df_year.columns = df_year.iloc[2]
        df_year = df_year.drop(['점유율 (%)'], axis=1)
        df_year = df_year.drop(2, axis=0)

        # drop '가정용', '공공용'
        df_year.columns = df_year.iloc[0]
        df_year = df_year.drop(['가정용', '공공용'], axis=1)

        # drop '농림수산업', '광업'
        df_year.columns = df_year.iloc[1]
        df_year = df_year.drop(['농림수산업', '광업'], axis=1)

        df_year.columns = ['month', 'total', 'service', 'a', 'manufact', 'year']
        df_year = df_year.drop(['a'], axis=1)
        df_year = df_year.drop([0, 1], axis=0).reset_index(drop=True)
        
        df_year['month'] = [re.sub("[^0-9]", "", row) for row in df_year['month']]
        df_year['datetime'] = pd.to_datetime(df_year.year + " " + df_year.month, format='%Y-%m')
        df_year['datetime'] = df_year['datetime'].astype(str).str.slice(stop=7)
        df_year = df_year.set_index('datetime').drop(['month', 'year'], axis=1)
        power_use = pd.concat([power_use,df_year])
    
    # start~end
    term = power_use.reset_index()
    start = term.loc[term['datetime']==start_year_month].index.values
    end = term.loc[term['datetime']==end_year_month].index.values
    result = term.iloc[int(start):int(end)+1]
        
    return result

In [10]:
power_use = term_useage(df,'2017-07','2018-11') 
power_use

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 53.60it/s]


,datetime,total,service,manufact
54,2017-07,4240321,522746,3267959
55,2017-08,4316873,563067,3211922
56,2017-09,4109028,522025,3114607
57,2017-10,3903712,493195,3004297
58,2017-11,4263161,568208,3126706
59,2017-12,4437240,726144,3171125
60,2018-01,4628993,795754,3245133
61,2018-02,4387240,840162,2944161
62,2018-03,4370210,666288,3182851
63,2018-04,4158537,582306,3101048
